# Transfer Learning

В этом блокноте вы научитесь использовать предобученные сети для решения задач в области компьютерного зрения. В частности, вы будете использовать сети, обученные на [ImageNet](http://www.image-net.org/) [доступно из torchvision](https://pytorch.org/vision/main/models.html). 

ImageNet — это обширный набор данных с более чем 1 миллионом размеченных изображений и 1000 категорий. Он используется для обучения глубоких свёрточных нейронных сетей. Свёрточные сети мы подробнее рассмотрим на следующей лабораторной.

После обучения эти модели работают удивительно хорошо для извлечения признаков из изображений, на которых они не были обучены. Использование заранее обученной сети на изображениях, которых нет в обучающем наборе, называется transfer learning (перенос обучения). Здесь мы будем использовать transfer learning, чтобы обучить сеть, которая может классифицировать фотографии кошек и собак с почти идеальной точностью.

С помощью `torchvision.models` вы можете загрузить эти предобученные сети и использовать их в своих приложениях. Добавим `models` в наши импорты.

In [1]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt

import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models

c:\Users\user1\anaconda3\envs\YOLO\Lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: 'Could not find module 'C:\Users\user1\anaconda3\envs\YOLO\Lib\site-packages\torchvision\image.pyd' (or one of its dependencies). Try using the full path with constructor syntax.'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


Большинство предобученных моделей требуют, чтобы входные данные были изображениями размером 224x224. Также нам нужно использовать ту же нормализацию, что использовалась при обучении моделей. Каждый цветовой канал нормализовался отдельно, средние значения составляют `[0.485, 0.456, 0.406]`, а стандартные отклонения — `[0.229, 0.224, 0.225]`.

In [2]:
## TODO

data_dir = 'Cat_Dog_data'

# TODO: Определите преобразования для обучающего и тестового наборов данных
train_transforms = transforms.Compose([transforms.Resize(255),
                                        transforms.RandomRotation(45),
                                        transforms.RandomResizedCrop(224),
                                        transforms.RandomHorizontalFlip(0.5),
                                        transforms.ToTensor(),
                                        transforms.Normalize([0.485, 0.456, 0.406], 
                                                            [0.229, 0.224, 0.225],
                                                            inplace = True)
                                        ])

test_transforms = transforms.Compose([transforms.Resize(224),
                                        transforms.CenterCrop(224),
                                        transforms.ToTensor()
                                    ])


# Примените преобразования, создайте объекты датасетов и загрузчиков данных для обучающего и тестового наборов
train_data = datasets.ImageFolder(data_dir + '/train', transform=train_transforms)
test_data = datasets.ImageFolder(data_dir + '/test', transform=test_transforms)

trainloader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True)
testloader = torch.utils.data.DataLoader(test_data, batch_size=64)

Загрузим модель, такую как [DenseNet](https://pytorch.org/vision/main/models/generated/torchvision.models.densenet121.html). Давайте выведем архитектуру модели, чтобы увидеть ее составные блоки.

In [3]:
model = models.densenet121(pretrained=True)
model

c:\Users\user1\anaconda3\envs\YOLO\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\user1\anaconda3\envs\YOLO\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet121_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/densenet121-a639ec97.pth" to C:\Users\user1/.cache\torch\hub\checkpoints\densenet121-a639ec97.pth
100%|██████████| 30.8M/30.8M [00:00<00:00, 48.7MB/s]


DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu

Эта модель состоит из двух основных частей: "извлекателя" признаков и классификатора. "Извлекатель" признаков — это стек свёрточных слоёв, который в целом формирует набор признаков, который можно передать классификатору. Часть классификатора — это один полносвязный слой `(classifier): Linear(in_features=1024, out_features=1000)`. Этот слой был обучен на наборе данных ImageNet, поэтому он не подойдёт для нашей конкретной задачи. Это означает, что нам нужно заменить классификатор, но признаки будут работать идеально сами по себе. В общем, заранее обученные сети можно понимать как удивительно хорошие детекторы признаков, которые могут использоваться в качестве входных данных для простых классификаторов.

In [4]:
# Замораживаем параметры, чтобы мы не могли выполнить обратное распространение ошибки через них
for param in model.parameters():
    param.requires_grad = False

from collections import OrderedDict
classifier = nn.Sequential(OrderedDict([
                          ('fc1', nn.Linear(1024, 500)),
                          ('relu', nn.ReLU()),
                          ('fc2', nn.Linear(500, 2)),
                          ('output', nn.LogSoftmax(dim=1))
                          ]))
    
model.classifier = classifier

Затем нам необходимо обучить классификатор. Однако теперь мы используем **очень глубокую** нейронную сеть. Если вы попытаетесь обучить её на центральном процессоре, как обычно, это займет очень много времени. Вместо этого мы будем использовать графический процессор (GPU) для выполнения вычислений. Вычисления линейной алгебры выполняются параллельно на GPU, что приводит к увеличению скорости обучения в 100 раз. Также возможно обучение на нескольких GPU, что ещё больше сокращает время обучения.

PyTorch, наряду с практически всеми другими фреймворками глубокого обучения, использует [CUDA](https://developer.nvidia.com/cuda-zone) для эффективного выполнения прямых и обратных проходов на GPU. В PyTorch вы перемещаете параметры вашей модели и другие тензоры в память GPU, используя `model.to('cuda')`. Вы можете перемещать их обратно с GPU с помощью `model.to('cpu')`, что вам часто нужно делать, когда вам нужно работать с выходом сети вне PyTorch. В качестве демонстрации увеличенной скорости сравним, сколько времени требуется для выполнения прямого и обратного прохода с помощью и без графического процессора.

In [5]:
import time

In [6]:
torch.cuda.is_available()

False

In [7]:
for device in ['cpu']:

    criterion = nn.NLLLoss()
    # Обучаем только параметры классификатора, параметры извлечения признаков заморожены
    optimizer = optim.Adam(model.classifier.parameters(), lr=0.001)

    model.to(device)

    for ii, (inputs, labels) in enumerate(trainloader):

        # Перемещаем тензоры входных данных и меток на GPU
        inputs, labels = inputs.to(device), labels.to(device)

        start = time.time()

        outputs = model.forward(inputs)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if ii==3:
            break
        
    print(f"Device = {device}; Time per batch: {(time.time() - start)/3:.3f} seconds")

Device = cpu; Time per batch: 6.773 seconds


Возможно писать код, не зависимый от устройства, который будет автоматически использовать CUDA, если оно включено, следующим образом:
```python
# в начале скрипта
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

...

# а затем каждый раз, когда вы получаете новый Тензор или Модуль, выполните следующий вызов
# это не будет приводить к копированию, если они уже находятся на желаемом устройстве
input = data.to(device)
model = MyModule(...).to(device)
```

Закончите обучение модели самостоятельно. Процесс такой же, как и раньше, за исключением того, что теперь ваша модель намного мощнее. Вы должны легко получить точность выше 95%.

>**Упражнение:** Обучите предобученную модель на классификацию изображений кошек и собак. Продолжайте работать с моделью DenseNet или попробуйте ResNet — это тоже хорошая модель для первого ознакомления. Убедитесь, что вы обучаете только классификатор, а параметры извлечения признаков заморожены.

In [22]:
## TODO
model = models.densenet121(pretrained=True)

for param in model.parameters():
    param.requires_grad = False

classifier = nn.Sequential(OrderedDict([
                          ('fc1', nn.Linear(1024, 500)),
                          ('relu', nn.ReLU()),
                          ('fc2', nn.Linear(500, 2)),
                          ('output', nn.LogSoftmax(dim=1))
                          ]))
    
model.classifier = classifier

c:\Users\user1\anaconda3\envs\YOLO\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\user1\anaconda3\envs\YOLO\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet121_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [25]:
criterion = nn.NLLLoss()

optimizer = optim.Adam(model.classifier.parameters(), lr=0.001)

epochs = 1

train_losses, val_losses, train_acc, val_acc = [], [], [], []

for e in range(epochs):
    model.classifier.train()
    accuracy = 0
    running_loss = 0
    for images, labels in trainloader:
        optimizer.zero_grad()
        log_ps = model.forward(images)
        loss = criterion(log_ps, labels)
        loss.backward()
        optimizer.step()
        ps = torch.exp(log_ps)
        top_p, top_class = ps.topk(1, dim=1)
        equals = top_class == labels.view(*top_class.shape)
        acc = torch.mean(equals.type(torch.FloatTensor)).item()
        accuracy += acc
        running_loss += loss.item()
        print(f'loss: {round(loss.item(),3)}, Accuracy {round(acc,3)}')
    accuracy /= len(trainloader)
    train_acc.append(accuracy)
    train_losses.append(running_loss/len(trainloader))

    with torch.no_grad():
        model.eval()
        accuracy = 0
        total_loss = 0
        for images, labels in testloader:
            log_ps = model.forward(images)
            loss = criterion(log_ps, labels)
            total_loss += loss.item()
            ps = torch.exp(log_ps)
            top_p, top_class = ps.topk(1, dim=1)
            equals = top_class == labels.view(*top_class.shape)
            accuracy = torch.mean(equals.type(torch.FloatTensor)).item()
        accuracy /= len(testloader)
        val_acc.append(accuracy)
        val_losses.append(total_loss/len(testloader))
            
    print(f'epoch {e+1}/{epochs} : loss = {round(train_losses[-1],3)}, val_loss = {round(val_losses[-1],3)}, train_acc: {round(train_acc[-1],3)}, val_acc: {round(val_acc[-1],3)}')

loss: 0.878, Accuracy 0.406
loss: 0.808, Accuracy 0.578
loss: 0.986, Accuracy 0.484
loss: 0.661, Accuracy 0.484
loss: 0.515, Accuracy 0.875
loss: 0.636, Accuracy 0.516
loss: 0.6, Accuracy 0.531
loss: 0.524, Accuracy 0.656
loss: 0.477, Accuracy 0.766
loss: 0.487, Accuracy 0.812
loss: 0.49, Accuracy 0.703
loss: 0.421, Accuracy 0.859
loss: 0.405, Accuracy 0.875
loss: 0.393, Accuracy 0.875
loss: 0.358, Accuracy 0.891
loss: 0.336, Accuracy 0.953
loss: 0.33, Accuracy 0.906
loss: 0.384, Accuracy 0.844
loss: 0.259, Accuracy 0.922
loss: 0.327, Accuracy 0.844
loss: 0.393, Accuracy 0.797
loss: 0.28, Accuracy 0.922
loss: 0.248, Accuracy 0.922
loss: 0.297, Accuracy 0.844
loss: 0.243, Accuracy 0.953
loss: 0.277, Accuracy 0.844
loss: 0.241, Accuracy 0.891
loss: 0.345, Accuracy 0.844
loss: 0.195, Accuracy 0.922
loss: 0.259, Accuracy 0.859
loss: 0.314, Accuracy 0.828
loss: 0.264, Accuracy 0.859
loss: 0.198, Accuracy 0.906
loss: 0.186, Accuracy 0.922
loss: 0.322, Accuracy 0.812
loss: 0.187, Accuracy 0.9